In [1]:
pip install  streamlit joblib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('CarPricesPrediction.csv', index_col=0)
df.reset_index(drop=True, inplace=True)

# Apply Label Encoding and save encoders
make_encoder = LabelEncoder()
model_encoder = LabelEncoder()
condition_encoder = LabelEncoder()

df["Make"] = make_encoder.fit_transform(df["Make"])
df["Model"] = model_encoder.fit_transform(df["Model"])
df["Condition"] = condition_encoder.fit_transform(df["Condition"])

# Save the encoders
joblib.dump(make_encoder, "make_encoder.pkl")
joblib.dump(model_encoder, "model_encoder.pkl")
joblib.dump(condition_encoder, "condition_encoder.pkl")

# Train Model
X = df.drop('Price', axis=1)
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Save the trained model
joblib.dump(rf_model, "Car_Prediction_Model.pkl")
print("✅ Model and encoders saved successfully!")


✅ Model and encoders saved successfully!


In [19]:
import streamlit as st
import numpy as np
import joblib

# Load trained model and encoders
model = joblib.load("Car_Prediction_Model.pkl")
make_encoder = joblib.load("make_encoder.pkl")
model_encoder = joblib.load("model_encoder.pkl")
condition_encoder = joblib.load("condition_encoder.pkl")

# Get original categories
make_categories = list(make_encoder.classes_)
model_categories = list(model_encoder.classes_)
condition_categories = list(condition_encoder.classes_)

# Streamlit UI
st.title("Car Price Prediction")
st.write("Enter car details to predict the price.")

# Input fields
make = st.selectbox("Select Make", make_categories)
model_name = st.selectbox("Select Model", model_categories)
year = st.number_input("Enter Year", min_value=2000, max_value=2025, value=2015)
mileage = st.number_input("Enter Mileage", min_value=0, max_value=300000, value=50000)
condition = st.selectbox("Select Condition", condition_categories)

if st.button("Predict Price"):
    # Encode categorical inputs using trained encoders
    try:
        make_encoded = make_encoder.transform([make])[0]
        model_encoded = model_encoder.transform([model_name])[0]
        condition_encoded = condition_encoder.transform([condition])[0]
    except ValueError as e:
        st.error(f"Encoding Error: {e}")
        st.stop()

    # Create input array
    input_features = np.array([[make_encoded, model_encoded, year, mileage, condition_encoded]])

    # Ensure input shape matches training data
    st.write(f"Encoded Input Features: {input_features}")

    # Predict price
    price = model.predict(input_features)
    st.write(f"Predicted Price: ${price[0]:,.2f}")


In [21]:
python CarPrediction.py


SyntaxError: invalid syntax (3923452300.py, line 1)

In [ ]:


!streamlit run app.py
